In [333]:
import sys
import cv2
import numpy as np
import pandas as pd
from cv2 import imread
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import time

import tensorflow as tf
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model

from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

from sklearn.utils import shuffle

import numpy.random as rng

In [334]:
save_path = "./savePath"

train_folder ="C:\\re-id datasets\\reid_dataset_trainvaltest\\train"
val_folder = "C:\\re-id datasets\\reid_dataset_trainvaltest\\validation"

In [335]:
def loadimgs(path, n = 0):
    X=[]
    y = []  
    class_dict = {}
    curr_y = n
    for identity in os.listdir(path):
        print(f"Loading class: {identity}")

        class_dict[identity] = [curr_y,None]

        print("Constructing class path")
        identity_path = os.path.join(path,identity)
        #Create an array with all images for this identity
        identity_images=[]

        for img in os.listdir(identity_path):

            #Create path for current image and load it
            image_path = os.path.join(identity_path, img)
            image = imread(image_path)
            #Add image to class array
            identity_images.append(image)
            y.append(curr_y)

        try:
            X.append(np.stack(identity_images))
        # edge case  - last one
        except ValueError as e:
            print(e)
            print("error - identity_images:", identity_images)

        curr_y += 1
        class_dict[identity][1] = curr_y - 1

    y = np.vstack(y)
    X = np.stack(X)
    return X,y,class_dict

   

In [336]:
X,y,c=loadimgs(train_folder)


print(f"X shape: {X.shape} Y shape: {y.shape} Class dict: {c}")


Loading class: 1
Constructing class path
Loading class: 10
Constructing class path
Loading class: 11
Constructing class path
Loading class: 12
Constructing class path
Loading class: 13
Constructing class path
Loading class: 14
Constructing class path
Loading class: 2
Constructing class path
Loading class: 3
Constructing class path
Loading class: 4
Constructing class path
Loading class: 5
Constructing class path
Loading class: 6
Constructing class path
Loading class: 7
Constructing class path
Loading class: 8
Constructing class path
Loading class: 9
Constructing class path
X shape: (14, 494, 128, 64, 3) Y shape: (6916, 1) Class dict: {'1': [0, 0], '10': [1, 1], '11': [2, 2], '12': [3, 3], '13': [4, 4], '14': [5, 5], '2': [6, 6], '3': [7, 7], '4': [8, 8], '5': [9, 9], '6': [10, 10], '7': [11, 11], '8': [12, 12], '9': [13, 13]}


In [337]:
with open(os.path.join(save_path,"train.pickle"), "wb") as f:
    pickle.dump((X,c),f)

In [338]:
Xval,yval,cval=loadimgs(val_folder)

Loading class: 1
Constructing class path
Loading class: 10
Constructing class path
Loading class: 11
Constructing class path
Loading class: 12
Constructing class path
Loading class: 13
Constructing class path
Loading class: 14
Constructing class path
Loading class: 2
Constructing class path
Loading class: 3
Constructing class path
Loading class: 4
Constructing class path
Loading class: 5
Constructing class path
Loading class: 6
Constructing class path
Loading class: 7
Constructing class path
Loading class: 8
Constructing class path
Loading class: 9
Constructing class path


In [339]:
with open(os.path.join(save_path,"val.pickle"), "wb") as f:
    pickle.dump((Xval,cval),f)

In [340]:
def initialize_weights(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer weights with mean as 0.0 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.0, scale = 1e-2, size = shape)

In [341]:
def initialize_bias(shape, dtype=None):
    """
        The paper, http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
        suggests to initialize CNN layer bias with mean as 0.5 and standard deviation of 0.01
    """
    return np.random.normal(loc = 0.5, scale = 1e-2, size = shape)

In [342]:
def get_siamese_model(input_shape):
    """
        Model architecture based on the one provided in: http://www.cs.utoronto.ca/~gkoch/files/msc-thesis.pdf
    """
    
    # Define the tensors for the two input images
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Convolutional Neural Network
    model = Sequential()
    model.add(Conv2D(64, (10,10), activation='relu', input_shape=input_shape,
                   kernel_initializer=initialize_weights, kernel_regularizer=l2(2e-4),data_format='channels_last', padding='same'))
    model.add(MaxPooling2D(data_format='channels_last', padding='same'))
    model.add(Conv2D(128, (7,7), activation='relu',
                     kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4),data_format='channels_last', padding='same'))
    model.add(MaxPooling2D(data_format='channels_last', padding='same'))
    model.add(Conv2D(128, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4),data_format='channels_last', padding='same'))
    model.add(MaxPooling2D(data_format='channels_last', padding='same'))
    model.add(Conv2D(256, (4,4), activation='relu', kernel_initializer=initialize_weights,
                     bias_initializer=initialize_bias, kernel_regularizer=l2(2e-4),data_format='channels_last', padding='same'))
    model.add(Flatten())
    model.add(Dense(4096, activation='sigmoid',
                   kernel_regularizer=l2(1e-3),
                   kernel_initializer=initialize_weights,bias_initializer=initialize_bias))
    
    # Generate the encodings (feature vectors) for the two images
    encoded_l = model(left_input)
    encoded_r = model(right_input)
    
    # Add a customized layer to compute the absolute difference between the encodings
    L1_layer = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
    L1_distance = L1_layer([encoded_l, encoded_r])
    
    # Add a dense layer with a sigmoid unit to generate the similarity score
    prediction = Dense(1,activation='sigmoid',bias_initializer=initialize_bias)(L1_distance)
    
    # Connect the inputs with the outputs
    siamese_net = Model(inputs=[left_input,right_input],outputs=prediction)
    
    # return the model
    return siamese_net

In [343]:
model = get_siamese_model((128, 64 ,3))
model.summary()

Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_31 (InputLayer)           [(None, 128, 64, 3)] 0                                            
__________________________________________________________________________________________________
input_32 (InputLayer)           [(None, 128, 64, 3)] 0                                            
__________________________________________________________________________________________________
sequential_15 (Sequential)      (None, 4096)         135429440   input_31[0][0]                   
                                                                 input_32[0][0]                   
__________________________________________________________________________________________________
lambda_15 (Lambda)              (None, 4096)         0           sequential_15[0][0]       

In [344]:
optimizer = Adam(lr = 0.00006)
model.compile(loss="binary_crossentropy",optimizer=optimizer)

In [345]:
with open(os.path.join(save_path, "train.pickle"), "rb") as f:
    (Xtrain, train_classes) = pickle.load(f)
    

print("Training alphabets: \n")
print(list(train_classes.keys()))

Training alphabets: 

['1', '10', '11', '12', '13', '14', '2', '3', '4', '5', '6', '7', '8', '9']


In [346]:
with open(os.path.join(save_path, "val.pickle"), "rb") as f:
    (Xval, val_classes) = pickle.load(f)

print("Validation alphabets:", end="\n\n")
print(list(val_classes.keys()))

Validation alphabets:

['1', '10', '11', '12', '13', '14', '2', '3', '4', '5', '6', '7', '8', '9']


In [347]:
def get_batch(batch_size,s="train"):
    """Create batch of n pairs, half same class, half different class"""
    if s == 'train':
        X = Xtrain
        classes = train_classes
    else:
        X = Xval
        classes = val_classes
    n_classes, n_examples, w, h, _ = X.shape

    # randomly sample several classes to use in the batch
    classes = rng.choice(n_classes,size=(batch_size,))
    
    # initialize 2 empty arrays for the input image batch
    pairs=[np.zeros((batch_size, w, h, 3)) for i in range(2)]
    
    # initialize vector for the targets
    targets=np.zeros((batch_size,))
    
    # make one half of it '1's, so 2nd half of batch has same class
    targets[batch_size//2:] = 1
    for i in range(batch_size):
        category = classes[i]
        idx_1 = rng.randint(0, n_examples)
        pairs[0][i,:,:,:] = X[category, idx_1].reshape(w, h, 3)
        idx_2 = rng.randint(0, n_examples)
        
        # pick images of same class for 1st half, different for 2nd
        if i >= batch_size // 2:
            category_2 = category  
        else: 
            # add a random number to the category modulo n classes to ensure 2nd image has a different category
            category_2 = (category + rng.randint(1,n_classes)) % n_classes
        
        pairs[1][i,:,:,:] = X[category_2,idx_2].reshape(w, h, 3)
    

    return pairs, targets

In [348]:
def generate(batch_size, s="train"):
    """a generator for batches, so model.fit_generator can be used. """
    while True:
        pairs, targets = get_batch(batch_size,s)
        yield (pairs, targets)

In [349]:
def make_oneshot_task(N, s="val", language=None):
    """Create pairs of test image, support set for testing N way one-shot learning. """
    if s == 'train':
        X = Xtrain
        categories = train_classes
    else:
        X = Xval
        categories = val_classes
    n_classes, n_examples, w, h, _ = X.shape
    
    indices = rng.randint(0, n_examples,size=(N,))
    if language is not None: # if language is specified, select characters for that language
        low, high = categories[language]
        if N > high - low:
            raise ValueError("This language ({}) has less than {} letters".format(language, N))
        categories = rng.choice(range(low,high),size=(N,),replace=False)

    else: # if no language specified just pick a bunch of random letters
        categories = rng.choice(range(n_classes),size=(N,),replace=False)            
    true_category = categories[0]
    ex1, ex2 = rng.choice(n_examples,replace=False,size=(2,))
    test_image = np.asarray([X[true_category,ex1,:,:]]*N).reshape(N, w, h,1)
    support_set = X[categories,indices,:,:]
    support_set[0,:,:] = X[true_category,ex2]
    support_set = support_set.reshape(N, w, h,1)
    targets = np.zeros((N,))
    targets[0] = 1
    targets, test_image, support_set = shuffle(targets, test_image, support_set)
    pairs = [test_image,support_set]

    return pairs, targets

In [350]:
def test_oneshot(model, N, k, s = "val", verbose = 0):
    """Test average N way oneshot learning accuracy of a siamese neural net over k one-shot tasks"""
    n_correct = 0
    if verbose:
        print("Evaluating model on {} random {} way one-shot learning tasks ... \n".format(k,N))
    for i in range(k):
        inputs, targets = make_oneshot_task(N,s)
        probs = model.predict(inputs)
        if np.argmax(probs) == np.argmax(targets):
            n_correct+=1
    percent_correct = (100.0 * n_correct / k)
    if verbose:
        print("Got an average of {}% {} way one-shot learning accuracy \n".format(percent_correct,N))
    return percent_correct

In [351]:
# Hyper parameters
evaluate_every = 200 # interval for evaluating on one-shot tasks
batch_size = 32
n_iter = 20000 # No. of training iterations
N_way = 20 # how many classes for testing one-shot tasks
n_val = 250 # how many one-shot tasks to validate on
best = -1

In [352]:
model_path = './weights/'

In [353]:
print("Starting training process!")
print("-------------------------------------")
t_start = time.time()
for i in range(1, n_iter+1):
    (inputs,targets) = get_batch(batch_size)
    loss = model.train_on_batch(inputs, targets)
    if i % evaluate_every == 0:
        print("\n ------------- \n")
        print("Time for {0} iterations: {1} mins".format(i, (time.time()-t_start)/60.0))
        print("Train Loss: {0}".format(loss)) 
        val_acc = test_oneshot(model, N_way, n_val, verbose=True)
        model.save_weights(os.path.join(model_path, 'weights.{}.h5'.format(i)))
        if val_acc >= best:
            print("Current best: {0}, previous best: {1}".format(val_acc, best))
            best = val_acc

Starting training process!
-------------------------------------
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)
(14, 494, 128, 64, 3)


KeyboardInterrupt: 

In [ ]:
model.load_weights(os.path.join(model_path, "weights.20000.h5"))

In [ ]:
def nearest_neighbour_correct(pairs,targets):
    """returns 1 if nearest neighbour gets the correct answer for a one-shot task
        given by (pairs, targets)"""
    L2_distances = np.zeros_like(targets)
    for i in range(len(targets)):
        L2_distances[i] = np.sum(np.sqrt(pairs[0][i]**2 - pairs[1][i]**2))
    if np.argmin(L2_distances) == np.argmax(targets):
        return 1
    return 0

In [ ]:
def test_nn_accuracy(N_ways,n_trials):
    """Returns accuracy of NN approach """
    print("Evaluating nearest neighbour on {} unique {} way one-shot learning tasks ...".format(n_trials,N_ways))

    n_right = 0
    
    for i in range(n_trials):
        pairs,targets = make_oneshot_task(N_ways,"val")
        correct = nearest_neighbour_correct(pairs,targets)
        n_right += correct
    return 100.0 * n_right / n_trials

In [ ]:
ways = np.arange(1,20,2)
resume =  False
trials = 50

In [ ]:
val_accs, train_accs,nn_accs = [], [], []
for N in ways:    
    val_accs.append(test_oneshot(model, N, trials, "val", verbose=True))
    train_accs.append(test_oneshot(model, N, trials, "train", verbose=True))
    nn_acc = test_nn_accuracy(N, trials)
    nn_accs.append(nn_acc)
    print ("NN Accuracy = ", nn_acc)
    print("---------------------------------------------------------------------------------------------------------------")

In [ ]:
with open(os.path.join(save_path,"accuracies.pickle"), "wb") as f:
    pickle.dump((val_accs,train_accs,nn_accs),f)

In [ ]:
with open(os.path.join(save_path, "accuracies.pickle"), "rb") as f:
    (val_accs, train_accs, nn_accs) = pickle.load(f)

In [ ]:
def concat_images(X):
    """Concatenates a bunch of images into a big matrix for plotting purposes."""
    nc, h , w, _ = X.shape
    X = X.reshape(nc, h, w)
    n = np.ceil(np.sqrt(nc)).astype("int8")
    img = np.zeros((n*w,n*h))
    x = 0
    y = 0
    for example in range(nc):
        img[x*w:(x+1)*w,y*h:(y+1)*h] = X[example]
        y += 1
        if y >= n:
            y = 0
            x += 1
    return img

In [ ]:
def plot_oneshot_task(pairs):
    fig,(ax1,ax2) = plt.subplots(nrows=1, ncols=2)
    ax1.matshow(pairs[0][0].reshape(105,105), cmap='gray')
    img = concat_images(pairs[1])
    ax1.get_yaxis().set_visible(False)
    ax1.get_xaxis().set_visible(False)
    ax2.matshow(img,cmap='gray')
    plt.xticks([])
    plt.yticks([])
    plt.show()

In [ ]:
# Example of concat image visualization
pairs, targets = make_oneshot_task(16,"train","Sanskrit")
plot_oneshot_task(pairs)

In [ ]:
fig,ax = plt.subplots(1)
ax.plot(ways, val_accs, "m", label="Siamese(val set)")
ax.plot(ways, train_accs, "y", label="Siamese(train set)")
plt.plot(ways, nn_accs, label="Nearest neighbour")

ax.plot(ways, 100.0/ways, "g", label="Random guessing")
plt.xlabel("Number of possible classes in one-shot tasks")
plt.ylabel("% Accuracy")
plt.title("Omiglot One-Shot Learning Performance of a Siamese Network")
box = ax.get_position()
ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
inputs,targets = make_oneshot_task(20, "val", 'Oriya')
plt.show()

plot_oneshot_task(inputs)